In [156]:
import pandas as pd

In [157]:
#разбиение на исходные 4 класса
def make_class(df_string, target_list):
    for i in range(0,len(target_list)):
        if  target_list[i].lower() in df_string.lower():
            return i
    return -1 #не интересующие нас классы

In [158]:
df = pd.read_excel('D:/стажер/auto_complectations.xlsx')

Задача классификации нужно решать для каких то бизнес задач. И в зависимости от них, мы определяем как и методы М.О., так и метрики, чтобы понять, что бизнесу нужно от нас. В данной задачи у нас задача в вакуме, поэтому я принимаю решение, что нам по каким то причинам поступают характеристики машин без их названий и брендов. Ибо если у нас есть название модели, комплектация и прочее - то это задача не машинного обучения, а скорее задача линейная . Поэтому столбцы brand,model,complect,type_car - исключаем, дабы приблизить задачу к реальности. Предположим, что нам пришли какие то тех. характеристики без описания машины.
Остальные столбцы подготовим к анализу. Либо их превратим в классы,либо их разделим на 3 столбца.
Так же оставим в столбцах лишь числа.
С помощью df.info() просмотрел на наличие NAN элементов(если где то неправильно заджойнили), они отсуствуют

In [159]:
df = df.drop(columns=['brand','model','complect'])

In [160]:
df

,type_car,seats,doors,Двигатель,Мощность,Крутящий момент двигателя,Коробка передач,Привод,Разгон до сотни,Максимальная скорость,Расход топлива (л/100 км)\nгород / трасса / смешанный,Дорожный просвет,Габариты (длина × ширина × высота),Колёсная база,Объём багажника,Объём багажника максимальный,Объём топливного бака,Масса автомобиля
0,Седан класса C,5,4,бензиновый (1395 см³),150 л. с.,250 Н·м,роботизированная (7 ступеней),передний,8.2 секунды,224 км/ч,5.9 / 4.1 / 4.8,165 мм,4458 × 1796 × 1416,2637 мм,425 л,880 л,50 л,1320 кг
1,Седан класса C,5,4,бензиновый (1395 см³),150 л. с.,250 Н·м,роботизированная (7 ступеней),передний,8.2 секунды,224 км/ч,5.9 / 4.1 / 4.8,165 мм,4458 × 1796 × 1416,2637 мм,425 л,880 л,50 л,1320 кг
2,Седан класса C,5,4,бензиновый (1984 см³),190 л. с.,320 Н·м,роботизированная (7 ступеней),передний,6.8 секунды,250 км/ч,7.2 / 4.7 / 5.6,165 мм,4458 × 1796 × 1416,2637 мм,425 л,880 л,50 л,1395 кг
3,Седан класса C,5,4,бензиновый (1984 см³),190 л. с.,320 Н·м,роботизированная (7 ступеней),полный,6.2 секунды,242 км/ч,7.2 / 4.8 / 5.7,165 мм,4458 × 1796 × 1416,2637 мм,390 л,845 л,55 л,1465 кг
4,Седан класса C,5,4,бензиновый (1984 см³),190 л. с.,320 Н·м,роботизированная (7 ступеней),передний,6.8 секунды,242 км/ч,7.2 / 4.7 / 5.6,165 мм,4458 × 1796 × 1416,2637 мм,425 л,880 л,50 л,1395 кг
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1698,Большой кроссовер,5,5,дизельный (1969 см³),235 л. с.,480 Н·м,автоматическая (8 ступеней),полный,7.8 секунды,220 км/ч,6.4 / 5.5 / 5.8,237 мм,4950 × 2008 × 1776,2984 мм,721 л,1899 л,71 л,1969 кг
1699,Большой кроссовер,7,5,бензиновый (1969 см³),320 л. с.,400 Н·м,автоматическая (8 ступеней),полный,6.5 секунды,230 км/ч,0 / 0 / 8.3,237 мм,4950 × 2008 × 1776,2984 мм,692 л,1899 л,71 л,2004 кг
1700,Большой кроссовер,7,5,дизельный (1969 см³),235 л. с.,480 Н·м,автоматическая (8 ступеней),полный,7.8 секунды,220 км/ч,6.4 / 5.5 / 5.8,237 мм,4950 × 2008 × 1776,2984 мм,692 л,1899 л,71 л,2009 кг
1701,Большой кроссовер,7,5,гибридный (1969 см³),407 л. с.,640 Н·м,автоматическая (8 ступеней),полный,5.6 секунды,230 км/ч,0 / 0 / 2.1,237 мм,4950 × 2008 × 1776,2984 мм,640 л,1899 л,70 л,2350 кг


In [161]:
#Разделим столбец двигатель на два(тип двигателя и его обьем)
df['Тип двигателя'] = df.apply(lambda x: x.Двигатель.split()[0], axis=1)
df['Обьем двигателя'] = df.apply(lambda x: x.Двигатель.split()[1][1:], axis=1)
#Разделим столбец расход двигателя на 3 типа.
df['Топливо город'] = df.apply(lambda x: x['Расход топлива (л/100 км)\nгород / трасса / смешанный'].split()[0], axis=1)
df['Топливо трасса'] = df.apply(lambda x: x['Расход топлива (л/100 км)\nгород / трасса / смешанный'].split()[2], axis=1)
df['Топливо смешанный'] = df.apply(lambda x: x['Расход топлива (л/100 км)\nгород / трасса / смешанный'].split()[4], axis=1)
#Габариты (длина × ширина × высота).Тут есть два пути. Можем сделать 1 фичу Обьем(посчитав), либо разделим на 3 фичи.
#Количество данных позволяет использовать 3 фичи.
df['Длина'] = df.apply(lambda x: x['Габариты (длина × ширина × высота)'].split()[0], axis=1)
df['Ширина'] = df.apply(lambda x: x['Габариты (длина × ширина × высота)'].split()[2], axis=1)
df['Высота'] = df.apply(lambda x: x['Габариты (длина × ширина × высота)'].split()[4], axis=1)

df = df.drop(columns=['Двигатель', 'Расход топлива (л/100 км)\nгород / трасса / смешанный','Габариты (длина × ширина × высота)'])

In [162]:
#Убираем С.И. из столбцов. Предварительно проверяю столбцы с помощью value_counts() на наличие каких то аномальных названий
df['Мощность'] = df.apply(lambda x: x.Мощность.split()[0], axis=1)
df['Крутящий момент двигателя'] = df.apply(lambda x: x['Крутящий момент двигателя'].split()[0], axis=1)
df['Разгон до сотни'] = df.apply(lambda x: x['Разгон до сотни'].split()[0], axis=1)
df['Максимальная скорость'] = df.apply(lambda x: x['Максимальная скорость'].split()[0], axis=1)
df['Дорожный просвет'] = df.apply(lambda x: x['Дорожный просвет'].split()[0], axis=1)
df['Колёсная база'] = df.apply(lambda x: x['Колёсная база'].split()[0], axis=1)
df['Объём багажника'] = df.apply(lambda x: x['Объём багажника'].split()[0], axis=1)
df['Объём багажника максимальный'] = df.apply(lambda x: x['Объём багажника максимальный'].split()[0], axis=1)
df['Объём топливного бака'] = df.apply(lambda x: x['Объём топливного бака'].split()[0], axis=1)
df['Масса автомобиля'] = df.apply(lambda x: x['Масса автомобиля'].split()[0], axis=1)
#грубо говоря заполняем Nan'ы по смыслу(ибо если нет разгона до 100, то он равен нулю)
df['Разгон до сотни'] = df.apply(lambda x: 0 if x['Разгон до сотни'] == 'нет' else x['Разгон до сотни'], axis=1)

In [163]:
#Превращаем классы в столбцы бинарного вида
df = pd.get_dummies(df,columns=['Коробка передач','Привод','Тип двигателя'],drop_first=True)

Наша задача - решить проблему классификации на основе 4 классов. Датасет содержит куда больше классов и подклассов. Т.к. я не особо знаю эту предметную область, поэтому не могу сказать, например, что класс "Универсал" по сути является подклассом "Кроссоверов" и прочее, поэтому упрощаю себе задачу таким образом - вычленим только те машины, в названиях класса которых присуствуют наше условие и только. Остальные данные удаляем

In [164]:
target_list=['Седан', 'Хэтчбек' , 'Кроссовер', 'Внедорожник']
df['target'] = df.apply(lambda x: make_class(x.type_car, target_list), axis=1)
df = df.loc[df.target!=-1]
df = df.drop(columns=['type_car','Коробка передач_автоматическая (5 ступеней)']) # удаляем лишний столбец и столбец, где в результате образовались одни нули

По-хорошему, нужно оставить только сильные корреляции(>0.1 по модулю) и таким образом снизить вероятность переобучения нашей модели и избавить себя от лишнего поиска выбросов.
И вот тут нам нужно понимать нашу бизнес задачу. Какое оборудование будет использоваться? Сколько раз будет задействоваться наша модель? Будет ли переобучение нашей модели и как часто? Зачем нам МЛ?
В зависимости от ответов на эти вопросы меняется и характер анализа и выбор конкретной модели машинного обучения

In [165]:
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

X, y = df.drop('target', axis=1), df['target']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42) #42

scaler = preprocessing.StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

Т.к. я решил не заниматься отбором признаков и удалением выборсов из нашей модели, мой выбор метода машинного обучения падает на робастные модели, т.е. устойчивые к выбросам. Например бэгинг на решающих деревьях

In [166]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
folds = StratifiedKFold(n_splits=3,shuffle=True,random_state = 42)
clf = RandomForestClassifier()
parametrs = { 'n_estimators': range (10, 140, 30),
              'max_depth': range (1,13, 2),
              'min_samples_leaf': range (1,8),
              'min_samples_split': range (2,10,2) }
grid = GridSearchCV(clf, parametrs, cv=folds,verbose = 4,n_jobs=-1, scoring='f1_weighted').fit(X_train, y_train)

Fitting 3 folds for each of 840 candidates, totalling 2520 fits


In [167]:
y_pred_test = grid.predict(X_test)
y_pred_train = grid.predict(X_train)

По поводу метрики - выбор падает на самую сбаланированную метрику F1 score. Если бы у нас была бы задача по определению, например, конкретно каких то классов, то использовалась другая метрика для определения точности попадания например в класс или полноту попадания.

In [168]:
from sklearn.metrics import f1_score
train_score = f1_score(y_train, y_pred_train, average = 'weighted')
test_score = f1_score(y_test, y_pred_test, average = 'weighted')

print(f'TRAIN SCORE: {train_score}')
print(f'TEST SCORE : {test_score}')

TRAIN SCORE: 1.0
TEST SCORE : 1.0


О чем говорят нам показатели 1.0? Модель решила задачу, но скорее всего решила ее линейно, посредством мультиколлиниарности или сильной корреляции. Расмотрев пару зависимостей в df.corr() я увидел, что в приводе очень высокая зависимость. И из за этого рождается 2 возможных варианта.
1)У нас ошибка на стадии анализа данных, мы неправильно подготовили данные(например не убрали таргет из трейна)
2)Это задача не является задачей машинного обучения. 
Для того, чтобы двигаться дальше в этой задаче необходимо больше вводных данных о том, где она будет использоваться и как пораждаются данные, чтобы понять, что мы делаем.